# 1 Introducción

El siguiente cuaderno realiza la multiplicación matriz x vector, utilizando GPGPU. Para su resolución se utilizo la libreria **OpenCL**, que es una alternativa diferente a CUDA y permite aprovechar la potencia de lo procesadores gráficos para realizar operaciones intensas. 

Para la resolución se aplicaron conceptos de Lenguaje Python, OPENCL y el manejo de operaciones aritméticas entre matrices y vectores.

---
# 2 Armado del ambiente
Toma la direcciòn web de una imagen con  acceso público en internet, la deja disponible al contexto de ejecuciòn del cuaderno colab.

---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [1]:
!pip install pyopencl 

     |████████████████████████████████| 747kB 23.5MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=72cd43f0994a581407d62505c848e70a24f3104fcd0dbbdbc405c907edcfb9b1
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pytools


---
# 3 Desarrollo


In [3]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

N =   100#@param {type: "number"}

# --------------------------------------------

import pyopencl as cl
from pyopencl import cltypes
import numpy
from datetime import datetime

tiempo_total = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# --------------------------------------------
if __name__ == "__main__":

    # Defino la memoria para el vectores en cpu.
    vector = numpy.random.randn( N )
    vector = vector.astype( numpy.float32() )

    matrix = numpy.random.randn( N , N )
    matrix = matrix.astype( numpy.float32() )

    tiempo_ini_cpu = datetime.now()

    #Obtengo plataforma OpenCL
    platform = cl.get_platforms()[0]
     
    #Obtengo una identificación de dispositivo (acelerador)
    device = platform.get_devices()[0]

    #Creo un contexto para el dispositivo seleccionado
    context = cl.Context([device])
     
    # CPU - Defino la función kernel que ejecutará en GPU.
    program = cl.Program(context, """
        __kernel void MatrizxVector(__global const float4 *matrix,
        __global const float4 *vector, __global float *result)
        {
          int gid = get_global_id(0);
          result[gid] = dot(matrix[gid], vector[0]);
        }
        """).build()
     
    tiempo_gpu = datetime.now()
    
    #Creo una cola de comandos para el dispositivo de destino
    queue = cl.CommandQueue(context)
     
    #Asigno la memoria del dispositivo y muevo los datos de entrada del host a la memoria del dispositivo.
    mem_flags = cl.mem_flags
    matrix_buf = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf=matrix)
    vector_buf = cl.Buffer(context, mem_flags.READ_ONLY | mem_flags.COPY_HOST_PTR, hostbuf=vector)
    matriz_x_vector = numpy.zeros(N, numpy.float32)
    destination_buf = cl.Buffer(context, mem_flags.WRITE_ONLY, matriz_x_vector.nbytes)
     
    tiempo_gpu = datetime.now()

    #Ejecución de Kernel
    program.MatrizxVector(queue, matriz_x_vector.shape, None, matrix_buf, vector_buf, destination_buf)
    
    tiempo_gpu = datetime.now() - tiempo_gpu

    #Muevo los datos de salida del kernel a la memoria del host.
    cl.enqueue_copy(queue, matriz_x_vector, destination_buf)
     
    #"""
    # CPU - Informo el resutlado.
    print( "------------------------------------")
    print( "VECTOR: " )
    print( vector )
    print( "------------------------------------")
    print( "MATRIZ: " )
    print( matrix )
    print( "------------------------------------")
    print( "Multiplicación: " )
    print( matriz_x_vector )
    print( "------------------------------------")
    #"""

    tiempo_total = datetime.now() - tiempo_total

    print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
    print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )


------------------------------------
VECTOR: 
[-0.16755164  0.88721204 -0.3626479  -0.11519394 -0.9628378  -0.9337501
 -0.07733484  0.1908773   0.39765906 -0.20686077 -1.0749935   0.81192875
  0.71897304 -2.4673293  -1.204745    0.5003272  -0.62082136 -1.9583313
 -0.24190967  0.1916502   0.33789918 -0.14364009  0.49533272  1.3450333
  0.8028735  -0.67952836 -1.2838707  -2.349502   -2.2026432   0.8174451
 -0.7803032  -0.5160703   0.5911788  -0.6622259   0.947358   -0.45198375
 -0.90028185  0.7177524   0.8838859  -0.562218   -0.849144    0.37244004
 -0.687453    0.70781505  0.9502905   0.01143862 -0.1509025  -1.1597444
 -1.9189411  -0.62214804 -0.43361112  0.37938195 -0.3491624  -0.18430084
  1.3107641  -0.21946192 -0.09654064  0.4740356  -0.58774287  0.96957654
 -0.99684304 -0.8162204  -0.69454443 -1.8997283   0.00601319 -1.1018468
  0.5139873  -1.5555272   0.10036635  0.36689243  0.13025744  0.7910716
  1.5515996  -0.45267537  0.92881364 -0.5239982  -1.2072972   0.48733404
 -1.7473189 

---
# 4 Tabla de pasos

Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de elementos N desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa el vector y matriz.
CPU  |  cl.get_platforms()[0]      | Obtengo plataforma OpenCL.
CPU  |  platform.get_devices()[0]   | Obtengo una identificación de dispositivo (acelerador)
CPU  |  cl.Context([device])   | Creo un contexto para el dispositivo seleccionado.
**GPU**      |  cl.Program        | Define el código del kernel 
CPU     |  cl.CommandQueue(context) | Creo una cola de comandos para el dispositivo de destino
CPU      |  cl.mem_flags/cl.Buffer        | Asigno la memoria del dispositivo y muevo los datos de entrada del host a la memoria del dispositivo.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cl.enqueue_copy   | Muevo los datos de salida del kernel a la memoria del host.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

Utilizar GPU con OpenCL reduce y optimiza los tiempos de ejecución de nuestro algoritmo si lo comparamos con el mismo programa ejecutado de forma secuencial que utiliza solo CPU. 
Para realizar este ejercicio elegí el lenguaje OpenCL ya que me parecio interesante porque presenta una serie de ventajas que no nos ofrece CUDA. En primer CUDA es un lenguaje pensado estrictamente para GPU y que actualmente solo funciona sobre tarjetas Nvidia (el propietario del lenguaje), mientras que OpenCL funciona tanto en tarjetas ATI como en tarjetas Nvidia y otros dispositivos diferentes, como por ejemplo una CPU convencional, siempre que se usen los drivers apropiados.Otro punto a favor de OpenCL es que, como su nombre indica, fomenta el codigo abierto y es de libre distribución. 

**Sugerencias de mejora:** 

1) Como nuestro algoritmo solo funciona con matrices cuadradas se podria realizar una mejora que realice la validación de si es posible multiplicar una determinada matriz con un vector. Ejemplo:
A es matriz de 3X2 y  B es vector de 2X1. En este caso se pueden multiplicar porque el número de columnas de A es igual al número de filas de B. 
 

---
# 6 Bibliografía

Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

Referencias: [WEB]

https://ecatue.gitlab.io/gpu2018/pages/Cookbook/matrix_multiplication_opencl.html

https://en.wikipedia.org/wiki/OpenCL
